## Analysis by tabularisai

In [1]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
model_name = "tabularisai/multilingual-sentiment-analysis"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# 推論関数を定義 (例)
def predict_sentiment(text: str):
    inputs = tokenizer(text, return_tensors="pt", truncation=True, padding=True).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
    logits = outputs.logits
    predicted_id = torch.argmax(logits, dim=-1).item()
    return predicted_id

# テスト推論
sample_text = "I love this product. It's really amazing!"
sentiment = predict_sentiment(sample_text)
print("Predicted label ID:", sentiment)

/Users/john/anaconda3/envs/Analysis/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Predicted label ID: 4


In [7]:
print(model.config.id2label)

{0: 'Very Negative', 1: 'Negative', 2: 'Neutral', 3: 'Positive', 4: 'Very Positive'}


In [2]:
# filepath: /path/to/SentimentAnalysis_tabularisai.py
import os
import glob
import pandas as pd
from transformers import pipeline

def analyze_sentiment_in_csvs(input_pattern: str):
    """
    指定したパターンにマッチするCSVファイルをすべて読み込み、
    'Detail'列 (index=1) に対して感情分析を行い、
    新たな列 'Sentiment-tabularisai' (ラベル) と 'Sentiment-tabularisai-score' (スコア) を
   追加して、同じファイル名で上書き保存する関数。

    Parameters:
    -----------
    input_pattern : str
        読み込むファイルパスのパターン。
        例: "root/Analyzed/AnalyzedData/analyzed_*_*.csv"
    """

    # 1. Sentiment Analysis パイプラインの準備
    #    多言語の感情分析を想定し、Hugging Faceから公開されているモデルを利用
    #    例: "tabularisai/multilingual-sentiment-analysis"
    sentiment_pipeline = pipeline(
        task="sentiment-analysis",
        model="tabularisai/multilingual-sentiment-analysis"
    )

    # 2. 指定パターンに合うCSVファイルをすべて取得
    csv_files = glob.glob(input_pattern)
    if not csv_files:
        print(f"パターン {input_pattern} に合致するCSVファイルがありません。処理を終了します。")
        return

    for csv_path in csv_files:
        print(f"=== 処理中: {csv_path} ===")

        # 3. CSVを読み込む
        df = pd.read_csv(csv_path)

        # dfのカラム構造:
        #   column[0]: Speaker
        #   column[1]: Detail
        #   column[2]: UniqueWords
        #   column[3]: JLPTLevel
        # → 新たに column[4]: Sentiment-tabularisai
        #               column[5]: Sentiment-tabularisai-score を追加

        # 4. Detail列に対して感情分析を適用
        labels = []
        scores = []
        for text in df["Detail"]:
            result = sentiment_pipeline(text)
            label = result[0]["label"]   # 例: "positive", "negative", "neutral"など
            score = result[0]["score"]   # 0.0～1.0の範囲

            labels.append(label)
            scores.append(score)

        # 5. 新たな列として追加
        df["Sentiment-tabularisai"] = labels
        df["Sentiment-tabularisai-score"] = scores

        # 6. 同じファイル名で上書き保存
        #    ※ 元データが必要な場合はバックアップを取るなどご注意ください
        df.to_csv(csv_path, index=False, encoding="utf-8-sig")
        
        print(f"上書き保存完了: {csv_path}\n")

if __name__ == "__main__":
    # 例: "root/Analyzed/AnalyzedData/analyzed_*_*.csv" で
    # analyzed_alpha_{name}.csv, analyzed_beta_{name}.csv を処理する想定。
    pattern = "./Analyzed/AnalyzedData/analyzed_*_*.csv"
    analyze_sentiment_in_csvs(input_pattern=pattern)

Device set to use mps:0


=== 処理中: ./Analyzed/AnalyzedData/analyzed_alpha_Hiraoka.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Hiraoka.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_alpha_Koga.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Koga.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_beta_Mori.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Mori.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_beta_Abe.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Abe.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_beta_Oyama.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Oyama.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_beta_Suto.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Suto.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_beta_Sakai.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Sakai.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_beta_Pang.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Pang.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_beta_Futam

In [6]:
import pandas as pd
file_path = './Analyzed/AnalyzedData/analyzed_*_*.csv'
def sentimentAnalyzeByTabularisai(file_path):
    file = pd.read_csv(file)
    df = pd.DataFrame(file)
    df = df.drop(columns=['Sentiment'])
    df.to_csv(file)

In [7]:
import pandas as pd
import glob
import os

# ファイルパスのパターンを指定
file_pattern = './Analyzed/AnalyzedData/analyzed_*_*.csv'

# ファイルパスのリストを取得
file_paths = glob.glob(file_pattern)

def sentimentAnalyzeByTabularisai(file_path):
    # CSVファイルを読み込む
    df = pd.read_csv(file_path)
    
    # 'Sentiment'列を削除する
    if 'Sentiment' in df.columns:
        df = df.drop(columns=['Sentiment'])
    
    # 元のファイルに上書き保存
    df.to_csv(file_path, index=False)

# すべてのファイルに対して処理を実行
for file_path in file_paths:
    sentimentAnalyzeByTabularisai(file_path)

## Analysis by lxyuan

In [8]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-classification", model="lxyuan/distilbert-base-multilingual-cased-sentiments-student")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
Device set to use mps:0


In [9]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("lxyuan/distilbert-base-multilingual-cased-sentiments-student")
model = AutoModelForSequenceClassification.from_pretrained("lxyuan/distilbert-base-multilingual-cased-sentiments-student")

In [29]:
# filepath: /path/to/SentimentAnalysis_lxyuan.py
import glob
import os
import pandas as pd
from transformers import pipeline

def analyze_with_lxyuan(input_pattern: str):
    """
    指定したパターンにマッチするCSVファイルをすべて読み込み、
    'Detail'列に対してlxyuanモデルで感情分析を行い、
    新たな列 'Sentiment-lxyuan' と 'Sentiment-lxyuan-score' を追加して上書き保存する関数。

    Parameters
    ----------
    input_pattern : str
        読み込むファイルパスのパターン (例: "./Analyzed/AnalyzedData/analyzed_*_*.csv")
    model_name : str
        使用するlxyuanモデル名。デフォルトは "lxyuan"
    """

    # 1. 感情分析パイプラインの準備
    #    "text-classification"タスクとして、lxyuanのモデルを使用
    sentiment_pipeline = pipeline(
        task="text-classification",
        model="lxyuan/distilbert-base-multilingual-cased-sentiments-student"
    )

    # 2. 指定パターンに合うCSVファイルを取得
    csv_files = glob.glob(input_pattern)
    if not csv_files:
        print(f"指定のパターン '{input_pattern}' に合致するCSVファイルがありません。処理を終了します。")
        return

    # 3. ファイルを順次読み込んで処理
    for csv_path in csv_files:
        print(f"=== 処理中ファイル: {csv_path} ===")
        df = pd.read_csv(csv_path)

        # 4. "Detail"列に対して感情分析を実施
        labels = []
        scores = []
        for text in df["Detail"]:
            result = sentiment_pipeline(text)
            # resultは [{'label': <str>, 'score': <float>}]
            label = result[0]["label"]   # ラベル名 (例: "POSITIVE", "NEGATIVE" など)
            score = result[0]["score"]   # 0.0 ～ 1.0 の範囲
            labels.append(label)
            scores.append(score)

        # 5. 新たな列として結果を追加
        df["Sentiment-lxyuan"] = labels
        df["Sentiment-lxyuan-score"] = scores

        # 6. 上書き保存 (バックアップが必要であれば別途コピーを推奨)
        df.to_csv(csv_path, index=False, encoding="utf-8-sig")
        print(f"上書き保存完了: {csv_path}\n")

if __name__ == "__main__":
    # フォルダ内のCSVを一括処理するパターンを指定
    pattern = "./Analyzed/AnalyzedData/analyzed_*_*.csv"
    # 関数を呼び出して処理を実行
    analyze_with_lxyuan(input_pattern=pattern)

Device set to use mps:0


=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_alpha_Hiraoka.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Hiraoka.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_beta_Mori.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Mori.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_beta_Abe.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Abe.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_beta_Suto.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Suto.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_beta_Sakai.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Sakai.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_beta_Pang.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Pang.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_beta_Futamura.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Futamura.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_alpha_Mori.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Mori.csv

=== 処理中ファイル: .

## jarvisx17/japanese-sentiment-analysis

In [30]:

# filepath: /path/to/SentimentAnalysis_jarvisx17.py
import glob
import os
import pandas as pd
from transformers import pipeline

def analyze_with_jarvisx17(input_pattern: str):

    # 1. 感情分析パイプラインの準備
    #    "text-classification"タスクとして、jarvisx17のモデルを使用
    sentiment_pipeline = pipeline(
        task="text-classification",
        model="jarvisx17/japanese-sentiment-analysis"
    )

    # 2. 指定パターンに合うCSVファイルを取得
    csv_files = glob.glob(input_pattern)
    if not csv_files:
        print(f"指定のパターン '{input_pattern}' に合致するCSVファイルがありません。処理を終了します。")
        return

    # 3. ファイルを順次読み込んで処理
    for csv_path in csv_files:
        print(f"=== 処理中ファイル: {csv_path} ===")
        df = pd.read_csv(csv_path)

        # 4. "Detail"列に対して感情分析を実施
        labels = []
        scores = []
        for text in df["Detail"]:
            result = sentiment_pipeline(text)
            # resultは [{'label': <str>, 'score': <float>}]
            label = result[0]["label"]   # ラベル名 (例: "POSITIVE", "NEGATIVE" など)
            score = result[0]["score"]   # 0.0 ～ 1.0 の範囲
            labels.append(label)
            scores.append(score)

        # 5. 新たな列として結果を追加
        df["Sentiment-jarvisx17"] = labels
        df["Sentiment-jarvisx17-score"] = scores

        # 6. 上書き保存 (バックアップが必要であれば別途コピーを推奨)
        df.to_csv(csv_path, index=False, encoding="utf-8-sig")
        print(f"上書き保存完了: {csv_path}\n")

if __name__ == "__main__":
    # フォルダ内のCSVを一括処理するパターンを指定
    pattern = "./Analyzed/AnalyzedData/analyzed_*_*.csv"
    # 関数を呼び出して処理を実行
    analyze_with_jarvisx17(input_pattern=pattern)

Device set to use mps:0


=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_alpha_Hiraoka.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Hiraoka.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_beta_Mori.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Mori.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_beta_Abe.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Abe.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_beta_Suto.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Suto.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_beta_Sakai.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Sakai.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_beta_Pang.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Pang.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_beta_Futamura.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Futamura.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_alpha_Mori.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Mori.csv

=== 処理中ファイル: .

##  Analysis by Mizuho-Sakura

In [4]:
# filepath: /path/to/SentimentAnalysis_MizuhoSakura.py
import glob
import os
import pandas as pd
from transformers import pipeline
def analyze_with_MizuhoSakura(input_pattern: str):

    # 1. 感情分析パイプラインの準備
    #    "text-classification"タスクとして、MizuhoSakuraのモデルを使用
    sentiment_pipeline = pipeline(
        task="text-classification",
        model="Mizuiro-sakura/luke-japanese-large-sentiment-analysis-wrime"
    )

    # 2. 指定パターンに合うCSVファイルを取得
    csv_files = glob.glob(input_pattern)
    if not csv_files:
        print(f"指定のパターン '{input_pattern}' に合致するCSVファイルがありません。処理を終了します。")
        return

    # 3. ファイルを順次読み込んで処理
    for csv_path in csv_files:
        print(f"=== 処理中ファイル: {csv_path} ===")
        df = pd.read_csv(csv_path)

        # 4. "Detail"列に対して感情分析を実施
        labels = []
        scores = []
        for text in df["Detail"]:
            result = sentiment_pipeline(text)
            # resultは [{'label': <str>, 'score': <float>}]
            label = result[0]["label"]   # ラベル名 (例: "POSITIVE", "NEGATIVE" など)
            score = result[0]["score"]   # 0.0 ～ 1.0 の範囲
            labels.append(label)
            scores.append(score)

        # 5. 新たな列として結果を追加
        df["Sentiment-MizuhoSakura"] = labels
        df["Sentiment-MizuhoSakura-score"] = scores

        # 6. 上書き保存 (バックアップが必要であれば別途コピーを推奨)
        df.to_csv(csv_path, index=False, encoding="utf-8-sig")
        print(f"上書き保存完了: {csv_path}\n")

if __name__ == "__main__":
    # フォルダ内のCSVを一括処理するパターンを指定
    pattern = "./Analyzed/AnalyzedData/analyzed_*_*.csv"
    # 関数を呼び出して処理を実行
    analyze_with_MizuhoSakura(input_pattern=pattern)

Device set to use mps:0


=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_alpha_Hiraoka.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Hiraoka.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_beta_Mori.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Mori.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_beta_Abe.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Abe.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_beta_Suto.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Suto.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_beta_Sakai.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Sakai.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_beta_Pang.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Pang.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_beta_Futamura.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Futamura.csv

=== 処理中ファイル: ./Analyzed/AnalyzedData/analyzed_alpha_Mori.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Mori.csv

=== 処理中ファイル: .

## Cristian phu

In [15]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_name = "christian-phu/bert-finetuned-japanese-sentiment"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
local_model_dir = "./Models/"

model.save_pretrained(local_model_dir)
tokenizer.save_pretrained(local_model_dir)

def analyze_with_Christian(input_pattern: str, local_model_dir):
    model = AutoModelForSequenceClassification.from_pretrained(local_model_dir)
    tokenizer = AutoTokenizer.from_pretrained(local_model_dir)
    
    sentiment_pipeline = pipeline(
        task="text-classification",
        model=model,
        tokenizer=tokenizer
    )

    csv_files = glob.glob(input_pattern)
    for csv_path in csv_files:
        df = pd.read_csv(csv_path)
        labels = []
        scores = []
        for text in df["Detail"]:
            result = sentiment_pipeline(text)
            label = result[0]["label"]
            score = result[0]["score"]
            labels.append(label)
            scores.append(score)
        
        df["Sentiment-Christian"] = labels
        df["Sentiment-Christian-score"] = scores
        
        df.to_csv(csv_path, index=False, encoding="utf-8-sig")
        print(f"上書き保存完了: {csv_path}\n")
        
if __name__ == "__main__":
    pattern = "./Analyzed/AnalyzedData/analyzed_*_*.csv"
    local_model_dir = "./Models/"
    analyze_with_Christian(input_pattern=pattern, local_model_dir=local_model_dir)

Device set to use mps:0


上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Hiraoka.csv

上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Mori.csv

上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Abe.csv

上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Suto.csv

上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Sakai.csv

上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Pang.csv

上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Futamura.csv

上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Mori.csv

上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Futamura.csv

上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Sakai.csv

上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Shiojiri.csv

上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Suto.csv

上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Hiraoka.csv

上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Abe.csv

上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Shiojiri.csv

上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Pang.csv



#### DataFrame確認用

In [20]:
import pandas as pd
file = pd.read_csv('./Analyzed/AnalyzedData/analyzed_alpha_abe.csv')
df = pd.DataFrame(file)
df 

,Speaker,Detail,WordsLen,UniqueWords,JLPTLevel,Sentiment-tabularisai,Sentiment-tabularisai-score,Sentiment-lxyuan,Sentiment-lxyuan-score,Sentiment-Christian,Sentiment-Christian-score
0,AI,こんにちは！最近、体の調子はどうですか？何か運動やリラクゼーションの活動をしていますか？それ...,74,36.000000,3.0,Very Negative,0.398403,positive,0.401744,positive,0.998923
1,User,最近は起きた時に寝ても寝ても眠い感じがする,21,13.000000,2.0,Negative,0.338300,negative,0.461021,positive,0.896546
2,AI,それはちょっと大変そうですね。睡眠の質が落ちている可能性があるかもしれませんね。寝る前の環境...,138,57.000000,2.0,Very Negative,0.561873,negative,0.730563,neutral,0.999262
3,User,スマホを見てしまうことなどは前からあったけど，前よりも疲れている感じがするんだよね,41,25.000000,2.0,Neutral,0.469847,negative,0.773605,neutral,0.969296
4,AI,疲れが溜まっていると感じるときは、やっぱり体も心も重くなりますよね。スマホの光が睡眠の質を下...,274,90.000000,2.0,Neutral,0.339138,positive,0.517148,neutral,0.800267
5,User,散歩するのに家の周りが面白くないんだけど，散歩が面白くなるアイデアが欲しいんだよな,41,18.000000,2.0,Neutral,0.397680,positive,0.456570,neutral,0.965423
6,AI,散歩をもっと楽しくするアイデア、いくつか提案しますね！ポッドキャストやオーディオブックを聴く...,449,126.000000,3.0,Positive,0.645495,positive,0.766437,positive,0.998347
7,AI Average,Mean Unique Words,17,77.250000,3.0,Neutral,0.424112,positive,0.511583,positive,0.994090
8,User Average,Mean Unique Words,17,18.666667,3.0,Neutral,0.424112,positive,0.511583,positive,0.994090


#### このファイルで行った感情分析Columnをリセットする場合

In [14]:
import os
import glob
import pandas as pd

def remove_columns_in_csvs(input_pattern: str):
    """
    指定したパターンにマッチするCSVファイルをすべて読み込み、
    指定した列（column 4以降）を削除して、同じファイル名で上書き保存する関数。

    Parameters:
    -----------
    input_pattern : str
        読み込むファイルパスのパターン。
        例: "root/Analyzed/AnalyzedData/analyzed_*_*.csv"
    """

    # 指定パターンに合うCSVファイルをすべて取得
    csv_files = glob.glob(input_pattern)
    if not csv_files:
        print(f"パターン {input_pattern} に合致するCSVファイルがありません。処理を終了します。")
        return

    for csv_path in csv_files:
        print(f"=== 処理中: {csv_path} ===")

        # CSVを読み込む
        df = pd.read_csv(csv_path)

        # 指定した列（column 4以降）を削除
        df = df.iloc[:, :8]

        # 同じファイル名で上書き保存
        df.to_csv(csv_path, index=False, encoding="utf-8-sig")
        
        print(f"上書き保存完了: {csv_path}\n")

if __name__ == "__main__":
    # 例: "root/Analyzed/AnalyzedData/analyzed_*_*.csv" で
    # analyzed_alpha_{name}.csv, analyzed_beta_{name}.csv を処理する想定。
    pattern = "./Analyzed/AnalyzedData/analyzed_*_*.csv"
    remove_columns_in_csvs(input_pattern=pattern)

=== 処理中: ./Analyzed/AnalyzedData/analyzed_alpha_Hiraoka.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Hiraoka.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_beta_Mori.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Mori.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_beta_Abe.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Abe.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_beta_Suto.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Suto.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_beta_Sakai.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Sakai.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_beta_Pang.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Pang.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_beta_Futamura.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_beta_Futamura.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_alpha_Mori.csv ===
上書き保存完了: ./Analyzed/AnalyzedData/analyzed_alpha_Mori.csv

=== 処理中: ./Analyzed/AnalyzedData/analyzed_alph